In [12]:
import os
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras import optimizers
from keras.models import load_model
from sklearn import metrics

np.random.seed(611)

In [13]:
# defining a window function for segmentation purposes
def windows(data, size):
    start = 0
    while start < data.count():
        yield int(start), int(start + size)
        start += (size/2)

####################### building the model #######################
def cnnModel():
    model = Sequential()
    model.add(Conv2D(128, (2,2), input_shape=(90,3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    adam = optimizers.Adam(lr=0.001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [14]:
####################### loading dataset #######################
filePath = r'G:\item\cnn_fig\terrain\test_L_H.csv'
columnNames = ['user_id', 'activity', 'timestamp', 'x-axis', 'y-axis', 'z-axis', 'low-terrain', 'high-terrain']
data = pd.read_csv(filePath, header=None, names=columnNames, na_values=';')

####################### data processing #######################
object = windows(data['timestamp'],90)
AllData = np.empty((0,90, 3, 3))
labels= np.empty((0))

for i in object:
    acc_x = data['x-axis'][i[0]:i[1]]
    acc_y = data['y-axis'][i[0]:i[1]]
    acc_z = data['z-axis'][i[0]:i[1]]
    terrain_l = data['low-terrain'][i[0]:i[1]]
    terrain_h = data['high-terrain'][i[0]:i[1]]
    
    if(len(data['timestamp'][i[0]:i[1]])==90):
        channel1 = np.vstack([np.dstack([acc_x, acc_y, acc_z])])
        channel2 = np.vstack([np.dstack([terrain_l]*3)])
        channel3 = np.vstack([np.dstack([terrain_h]*3)])

        AllChannel = np.stack((channel1,channel2,channel3), axis=-1)   
        AllChannel = np.reshape(AllChannel,(1,90,3,3))
        AllData = np.vstack([AllData,AllChannel])
  
        labels = np.append(labels, stats.mode(data['activity'][i[0]:i[1]])[0][0])

labels = np.asarray(pd.get_dummies(labels), dtype=np.int8) 
trainSplit = np.random.rand(len(AllData)) < 0.75 
trainX = AllData[trainSplit]
testX = AllData[~trainSplit]
trainX = np.nan_to_num(trainX)
testX = np.nan_to_num(testX)
trainY = labels[trainSplit]
testY = labels[~trainSplit]

####################### model trainning #######################
model = cnnModel()
for layer in model.layers:
    print(layer.name)
model.fit(trainX, trainY, validation_split=0.25, epochs=5, batch_size=64, verbose=2)
score = model.evaluate(testX, testY, verbose=2)
print('Accuracy: %.2f%%' %(score[1]*100))
model.save('CnnModel_terrain.h5')
model.save_weights('Cnn_Weigths.h5')
np.save('Cnn_output_3d.npy', testY)
np.save('Cnn_input_3d.npy', testX)

conv2d_4
max_pooling2d_4
dropout_4
flatten_4
dense_10
dense_11
dense_12
Train on 9015 samples, validate on 3006 samples
Epoch 1/5
 - 2s - loss: 0.4918 - accuracy: 0.8180 - val_loss: 0.0964 - val_accuracy: 0.9780
Epoch 2/5
 - 1s - loss: 0.1003 - accuracy: 0.9717 - val_loss: 0.0821 - val_accuracy: 0.9860
Epoch 3/5
 - 1s - loss: 0.0650 - accuracy: 0.9799 - val_loss: 0.0854 - val_accuracy: 0.9784
Epoch 4/5
 - 1s - loss: 0.0519 - accuracy: 0.9831 - val_loss: 0.1045 - val_accuracy: 0.9737
Epoch 5/5
 - 1s - loss: 0.0436 - accuracy: 0.9859 - val_loss: 0.0761 - val_accuracy: 0.9820
Accuracy: 98.54%
